<a href="https://colab.research.google.com/github/lilasch/Facial_Attribute_Classification/blob/main/liladatapreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Facial Attribute Classification**

Lila Schisgal and Phoebe Jeske


First, we import our data from two text files and partition it into training and validation data.

In [4]:
import zipfile
zippath = 'img_align_celeba.zip'
targetfolder = 'images'

with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(targetfolder)

In [1]:
from tensorflow import keras
import pandas as pd
import numpy as np
import glob
import shutil
import os


# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# 40 attributes in total
num_attributes = 40

# Load attributes csv
# attributes = pd.read_csv("/content/drive/MyDrive/list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")
attributes = pd.read_csv("list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")

# Load csv with partitions values
# partitions = pd.read_csv("/content/drive/MyDrive/list_eval_partition.txt", delimiter="\s+|\t", header = None)
partitions = pd.read_csv("list_eval_partition.txt", delimiter="\s+|\t", header = None)


partitions.columns = ['image_name', 'dataset'] # setting column header names for partitions
attributes['dataset'] = partitions['dataset'].values # copying the partition values into the attributes df

# filter so just the eyeglasses attribute is included
glasses_df = attributes.iloc[:, [15, num_attributes]]

# partitioning the data
train_df = glasses_df[glasses_df['dataset'] == 0]
valid_df = glasses_df[glasses_df['dataset'] == 1]
test_df = glasses_df[glasses_df['dataset'] == 2]

y_train = train_df['Eyeglasses']
y_valid = valid_df['Eyeglasses']
y_test = test_df['Eyeglasses']

# getting our image file names in two arrays
train_files = train_df.index
valid_files = valid_df.index
test_files = test_df.index

image_folder = "images/img_align_celeba/"
valid_destination = "valid_images"
train_destination = "train_images"
test_destination = "test_images"

# Function to split images based on classification
def split_images(source_folder):
    files = os.listdir(source_folder)  # Get the list of files in the source_folder
    for file in files:
        if file in train_files:
            if not os.path.exists(os.path.join(train_destination, file)):
                shutil.move(os.path.join(source_folder, file), os.path.join(train_destination, file))
        elif file in valid_files:
            if not os.path.exists(os.path.join(valid_destination, file)):
                shutil.move(os.path.join(source_folder, file), os.path.join(valid_destination, file))
        elif file in test_files:
            if not os.path.exists(os.path.join(test_destination, file)):
                shutil.move(os.path.join(source_folder, file), os.path.join(test_destination, file))
        else:
            continue

split_images(image_folder)

/var/folders/zz/p0bxtrc57nx_bmrp_9vns_r40000gn/T/ipykernel_1593/4022894428.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  attributes = pd.read_csv("list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")
/var/folders/zz/p0bxtrc57nx_bmrp_9vns_r40000gn/T/ipykernel_1593/4022894428.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  partitions = pd.read_csv("list_eval_partition.txt", delimiter="\s+|\t", header = None)


We then separate our target y values and image filenames in preparation for accessing the actual images as vectors.

Next: need to access the images from the dataset and put them into a set of vectors.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Set your folder paths
valid_folder = "valid_images/"
test_folder = "test_images/"
train_folder = "train_images/"

# Function to normalize images
def normalize_images(image_path):
    save_array = []
    for filename in os.listdir(image_path):
        if filename.endswith(".jpg"):
            img_path = os.path.join(image_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)
            flattened_dim = 218 * 178 * 3
            img_processed = img_array.reshape(-1, flattened_dim)
            img_processed = img_processed.astype(float)
            save_array.append(img_processed)
    return save_array

def standardize(train, test, valid):
    mean = np.mean(train)
    std = np.std(train)

    train = (train - mean)/std
    valid = (valid - mean)/std
    test = (test - mean)/std

    return train, test, valid



# Apply preprocessing steps
valid_images = normalize_images(valid_folder)
train_images = normalize_images(train_folder)
test_images = normalize_images(test_folder)

x_train, x_test, x_valid = standardize(train_images, test_images, valid_images)


block started
imported tensorflow
imported keras
imported pandas
Images normalized
valid_done


In [3]:
model = keras.Sequential()
model.add(keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.L2(0.01)))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

network_history_std = model.fit(x_train, y_train, validation_data=(x_valid,y_valid),
                                epochs=1000000000, callbacks=[early_stopping])

block started
tensorflow imported
keras imported
numpy imported
matplotlib imported


NameError: name 'x_train' is not defined

In [ ]:
eval = model.evaluate(x_test,y_test)
print("Test loss:",eval[0])
print("Test accuracy:",eval[1])